# The Curl Operator :
## Direct Problem
The problem is :given $\psi$, find $f$ such that:
$$
\begin{cases}
\text{curl } \psi = f & \text{in } \Omega\\
\left . f \right \vert_{\partial\Omega} = f_0 & \text{in } \partial\Omega
\end{cases}
$$

In the thesis is proven that the distributive $\text{curl}$ operator is given by:
$$

  \left\langle \text{curl } \psi, \phi \right\rangle = \int_{T_i} \psi \cdot \text{curl} \phi + \int_{\partial T_i} \phi\cdot n\times \psi  
$$

Obviously this holds true if $\psi$ is a well defined triangulation-wise with tangential continuity on the edges AND every edge integration must hold true! 

In [5]:
# some libraries we need

import time # for timing
from ngsolve import * 
from ngsolve.webgui import Draw 
from netgen.csg import unit_cube 
from ngsolve.krylovspace import CGSolver

import scipy.sparse as sp # for visualising the matrix
from scipy.sparse import csc_matrix 
import matplotlib.pylab as plt

In [6]:
n = specialcf.normal(3) # normal vector to a surface of a tetrahedron

def norm(u, Mesh):
    # norm of a scalar, vector or matrix function
    with TaskManager():
        return sqrt(Integrate( InnerProduct(u,u) , Mesh)) 

def Curl(u):
    # curl of a vector or matrix
    if u.dim == 3:
        return CF( (u[2].Diff(y)- u[1].Diff(z), u[0].Diff(z)- u[2].Diff(x), u[1].Diff(x)- u[0].Diff(y)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )

def Inc(u):
    # inc of a matrix
    return Curl((Curl(u)).trans)

P_n = OuterProduct(n,n)  # projection along normal direction
Q_n = Id(3) - OuterProduct(n,n) # projection along tangential direction
C_n = CF( (0,n[2],-n[1],-n[2],0,n[0],n[1],-n[0],0), dims=(3,3) ) # corss product perator with the normal vector

def C(V): 
    # corss product perator with the generic vector
    return CF( (0,V[2],-V[1],-V[2],0,V[0],V[1],-V[0],0), dims=(3,3) ) 


In [7]:
def distributive_curl(mesh,order = 1, psi = None, BND_Curl_psi = None ,draw = True, bonus_intorder = 15):
    fesHCurl = HCurl(mesh, order=order, dirichlet= ".*")
    
    ##########################################################
    #           Coefficient and grid functions               #
    ##########################################################

    gf_psi = GridFunction(fesHCurl) 
    gf_psi.Set ( psi, bonus_intorder = bonus_intorder, dual=True)

    gf_curl_psi = GridFunction(fesHCurl)
    gf_curl_psi.Set(BND_Curl_psi, definedon=mesh.Boundaries(".*"),  bonus_intorder = bonus_intorder, dual=True)


    ##########################################################
    #              Linear and Bilinear forms                 #
    ##########################################################

    u,v = fesHCurl.TnT()
    #some geometrical objects we need

    # Mass matrix
    a = BilinearForm(fesHCurl, symmetric=True)
    a += InnerProduct(u,v)*dx 

    # linear form induced by the metric gfG
    f = LinearForm(fesHCurl)

    # thetrahedron curl part
    f += InnerProduct(curl(gf_psi),v)*dx    

    # faces part:
    u_n = InnerProduct(gf_psi,n)
    f += InnerProduct(v,Cross(n,gf_psi))*dx(element_vb=BND)
    
    ##########################################################
    #           Assemble, compute and print!                 #
    ##########################################################
    pre = Preconditioner(a, "local")
    a.Assemble()
    f.Assemble()
    #plt.spy(csc_matrix(a.mat.CSR()))

    r = f.vec.CreateVector()
    r.data = f.vec - a.mat * gf_curl_psi.vec
        

    inverse = CGSolver(a.mat, pre.mat , printrates='\r', maxiter=1000,tol=1e-12)
    gf_curl_psi.vec.data += inverse * r
    #gfCurlCurlG.vec.data += a.mat.Inverse(freedofs=fesHCurlCurl.FreeDofs(),inverse="sparsecholesky") * r
    if draw:
        Draw(gf_psi, mesh)
        Draw(gf_curl_psi, mesh)

    return gf_psi , gf_curl_psi

# Example of direct problem

In [8]:

peak = exp(-25*((x-0.5)**2+(y-0.5)**2+(z-0.5)**2))

# our initail function
psi = CF ( (peak, 0, 0 ), dims=(3,1))

# curl that needs to be approx
curl_psi = CF( Curl(psi), dims=(3,1) )

Maxh = 0.2
Order = 3
mesh = Mesh(unit_cube.GenerateMesh(maxh=Maxh))
gf_psi , gf_curl_psi =  distributive_curl(mesh, order = Order,psi = psi, BND_Curl_psi= curl_psi, draw = False)

draw = True

if draw:
    print("psi : ")
    Draw(psi, mesh, clipping = (1,1,1))
    print("gf_psi : ")
    Draw(gf_psi, mesh,clipping = (1,1,1,0))
    print("Error psi-gf_psi "+ str(sqrt(Integrate(InnerProduct(psi-gf_psi,psi-gf_psi),mesh))))
    Draw(psi-gf_psi, mesh,clipping = (1,1,1,0))

    print("curl_psi : ")
    Draw(curl_psi, mesh,clipping = (1,1,1,0))
    print("gf_curl_psi : ")
    Draw(gf_curl_psi, mesh,clipping = (1,1,1,0))
    print("Error gf_curl_psi -curl_psi "+ str(sqrt(Integrate(InnerProduct(gf_curl_psi -curl_psi,curl_gf_psi -curl_psi),mesh))))
    Draw(gf_curl_psi -curl_psi, mesh,clipping = (1,1,1,0))


CG converged in 408 iterations to residual 1.5023850070536455e-12
psi : 


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

gf_psi : 


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

Error psi-gf_psi 0.0009926581482965287


...

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

curl_gf_psi : 


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

Error curl_gf_psi -curl_psi 0.024143622933362033


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…